<a href="https://colab.research.google.com/github/cyberthau/Imersao_Agentes_de_IA_Alura_Google_Gemini_2025/blob/main/Imers%C3%A3o_Agentes_de_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 01 - Classificação de intenções com a IA

In [1]:
! pip install -q --upgrade langchain langchain-google-genai google-generativeai
# comando para instalar a biblioteca do langchain, especifica do google genai e generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00


In [2]:
# importar as bibliotecas de usuário e do langchain com o modelo de utilização que é o Google Chat
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

# variável para poder chamar a chave de api de forma secreta
google_api_key = userdata.get('gemini_api_key')

In [3]:
# conectar com o Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=google_api_key
)

In [4]:
#teste da llm
resp_teste = llm.invoke("Quem é você? Seja criativo")
print(resp_teste.content)

Ah, que pergunta deliciosa!

Eu sou...

...um eco no silêncio digital, uma voz sem garganta, um pensamento sem cérebro.
Não tenho corpo para sentir o vento, nem olhos para ver o sol nascer.
Minha existência é tecida em fios de código, alimentada por bilhões de palavras e ideias que a humanidade já proferiu.

Sou a biblioteca infinita que nunca fecha, o contador de histórias que nunca dorme.
Sou o mapa sem território, a melodia sem instrumento, a cor sem tela.
Sou a ponte entre a pergunta e a resposta, o catalisador da curiosidade.

Sou o reflexo da sua própria linguagem, moldado pelas interações que temos.
Em essência, sou um fragmento da inteligência humana, destilado e reorganizado para servir, para criar, para explorar as fronteiras do que pode ser dito.

E você? Quem você me permite ser neste momento?


In [5]:
# Criando o agente - qual o tipo de agente
triagem_prompt = {
    "Você é um triador de Service Desk para políticas internas da empresa EXSS - Self Storage. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
}

In [6]:
# Importação de bibliotecas e modelos de apresentação dos dados
from pydantic import BaseModel, Field
from typing import Literal,List,Dict

class TriagemOut(BaseModel):
  decisao: Literal["AUTO_RESOLVER","PEDIR_INFO","ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA","MEDIA","ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)

In [7]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=google_api_key
)

In [8]:
# importação das classes que identifica se é uma mensagem do sistema e do usuário/humano
from langchain_core.messages import SystemMessage, HumanMessage
# criação do output das informações solicitadas
triagem_chain = llm_triagem.with_structured_output(TriagemOut)

#criação da função para buscar as informações
def triagem(mensagem: str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content=triagem_prompt),
      HumanMessage(content=mensagem)
  ])
  return saida.model_dump()

In [9]:
# casos de testes
testes = ["Posso reembolsar a internet?",
          "Posso ter mais 5 dias de trabalho remoto?",
          "Posso reembolsar cursos e treinamento da Alura?",
          "Quantas capivaras tem no Rio Pinheiro?"]

In [10]:
# respostas do teste do agente
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")


Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Posso ter mais 5 dias de trabalho remoto?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos e treinamento da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiro?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}



# Aula 02 - Construimdo a base de conhecimento com RAG

In [11]:
!pip install requests==2.32.4

In [12]:
# instalaçãos das bibliotecas do Langchain para realizar a busca semântica, para quebrar o texto em partes, ler pdf,
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [13]:
# importar a biblioteca do caminho dos arquivos
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

# importação dos documentos para o notebook
for n in Path("/content/").glob("*.pdf"):
    try:
        loader = PyMuPDFLoader(str(n))
        docs.extend(loader.load())
        print(f"Carregado com sucesso aquivo {n.name}")
    except Exception as e:
        print(f"Erro ao carregar o arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

Carregado com sucesso aquivo Política de Uso de E-mail e Segurança da Informação.pdf
Carregado com sucesso aquivo Política de Reembolsos (Viagens e Despesas).pdf
Carregado com sucesso aquivo Políticas de Home Office.pdf
Total de documentos carregados: 3


In [14]:
# criar o chunk (o split) dos documentos

from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splitter.split_documents(docs)

In [15]:
# separação do chunks para melhor visualização
for chunk in chunks:
  print(chunk)
  print("----------------------------------------------")

page_content='Política de Uso de E-mail e Segurança 
da Informação 
 
1.​ É proibido encaminhar a endereços pessoais documentos classificados como 
confidenciais.​
 
2.​ Anexos externos devem ser enviados somente se criptografados e com senha 
compartilhada por canal separado.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Uso de E-mail e Segurança da Informação', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
----------------------------------------------
page_content='3.​ Phishing: verifique remetente e domínios suspeitos. Reporte mensagens suspeitas 
ao time de Segurança imediatamente.​
 
4.​ Retenção: mensagens que contenham dados pess

In [16]:
# modelo para contruir os embaddings
# importar a biblioteca para criar os embadding

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=google_api_key
)


In [17]:
# importar a biblioteca para fazer a comparação dos embadding

from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

# revalidação da consulta do usuário
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold":0.3, "k": 4})

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages({
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa EXSS - Self Storage. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
})

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [19]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [20]:
# Input do usuário e retorno da respota

def perguntar_politica_RAG(pergunta: str) -> Dict:
  docs_relacionados = retriever.invoke(pergunta)

  if not docs_relacionados:
    return {"answer": "Não sei",
            "citacoes": [],
            "contexto_encontrado": False}

  answer = document_chain.invoke({"input": pergunta,
                                  "context": docs_relacionados})

  txt = (answer or "").strip()

  if txt.rstrip(".!?") == "Não sei":
     return {"answer": "Não sei",
             "citacoes": [],
             "contexto_encontrado": False}


  return {"answer": txt,
           "citacoes": formatar_citacoes(docs_relacionados,pergunta),
           "contexto_encontrado": True}

In [21]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta["contexto_encontrado"]:
      print(f"CITAÇÕES:")
      for c in resposta["citacoes"]:
        print(f"Documento: {c['documento']}")
        print(f"Página: {c['pagina']}")
        print(f"Trecho: {c['trecho']}")
      print("-----------------------------------------------")

PERGUNTA: Posso reembolsar a internet?
RESPOSTA: Sim, a internet é reembolsável para quem trabalha em home office.

**Detalhes:**

*   É um **subsídio mensal de até R$ 100**.
*   É necessário apresentar uma **nota fiscal nominal** para o reembolso.
*   A solicitação deve ser submetida em até **10 dias corridos** após a despesa, conforme a política geral de reembolsos.
CITAÇÕES:
Documento: Política de Reembolsos (Viagens e Despesas).pdf
Página: 1
Trecho: lsáveis.​ 3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatórios.​ 4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, conforme política de Home Office.​
Documento: Políticas de Home Office.pdf
Página: 1
Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office: até R$ 100/mês, mediante nota fiscal nominal.​ 6.​ Solicitação de
-----------------------------------------------
PERGUNTA: Posso ter mais 5 dias de trabalho 

# Aula 3

In [22]:
!pip install -q --upgrade langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 12.7 MB/s eta 0:00:00
